In [1]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as time
import math, cmath
import sys
import auxiliary_library as me

from IPython.display import display, Math, Latex

In [47]:
N = 7

spin_ops_list = me.one_body_spin_ops(N)
Heisenberg_Hamiltonian_tests(spin_ops_list, N)

magnetization = sum(spin_ops_list[3][a] for a in range(len(spin_ops_list[3])))

chain_type = "XXZ"; Hamiltonian_paras = [.2, .15, .1, 1.]
H_H = Heisenberg_Hamiltonian(spin_ops_list, chain_type, N, False, Hamiltonian_paras, False)
H_H

Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape = (128, 128), type = oper, isherm = True
Qobj data =
[[-9.89601686  0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -9.73893723 -0.31415927 ...  0.          0.
   0.        ]
 [ 0.         -0.31415927 -6.44026494 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  6.12610567 -0.31415927
   0.        ]
 [ 0.          0.          0.         ... -0.31415927  9.1106187
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   8.95353906]]

In [ ]:

me.commutator(H_H, magnetization)